# pagerank -> textrank
> python3

---
## data preprocessing

In [54]:
import sys
from pprint import pprint
import re
sys.version

'3.6.2 |Anaconda custom (64-bit)| (default, Sep 19 2017, 08:03:39) [MSC v.1900 64 bit (AMD64)]'

In [55]:
import math, random, csv, networkx as nx, operator
graph = nx.Graph()

with open('data/lesmis.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')

    data = [row[:3] for row in reader]
    nodes = set([row[0] for row in data])
    edges = [(row[0], row[2]) for row in data]
    num_nodes = len(nodes)
    rank = 1/float(num_nodes)
    graph.add_nodes_from(nodes, rank=rank)
    graph.add_edges_from(edges)

FileNotFoundError: [Errno 2] No such file or directory: 'data/lesmis.csv'

In [56]:
data

NameError: name 'data' is not defined

## text rank preprocessing

In [57]:
from gensim.summarization import summarize
from gensim.summarization import keywords
from gensim.summarization.textcleaner import split_sentences

import requests
from bs4 import BeautifulSoup
from pprint import pprint

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [63]:
text = requests.get(
#     "https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4(%EA%B8%B0%EC%97%85)"
    "http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=018&aid=0004079339"
#     "http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=105&oid=015&aid=0003923373"
)

In [64]:
# parse html
soup = BeautifulSoup(text.text, 'html.parser')

In [65]:
print(soup.prettify())

<!DOCTYPE HTML>
<html lang="ko">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=1106" name="viewport">
   <title>
    美컨슈머리포트 "삼성 갤럭시S9·S9+, 최고 스마트폰" : 네이버 뉴스
   </title>
   <meta content="이데일리 : 네이버뉴스" property="me2:post_tag">
    <meta content="이데일리" property="me2:category1"/>
    <meta content="IT/과학" property="me2:category2"/>
    <meta content="http://imgnews.naver.net/image/origin/018/2018/04/15/4079339.jpg" property="me2:image"/>
    <meta content='美컨슈머리포트 "삼성 갤럭시S9·S9+, 최고 스마트폰"' property="og:title"/>
    <meta content="article" property="og:type"/>
    <meta content="http://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;oid=018&amp;aid=0004079339&amp;sid1=001" property="og:url"/>
    <meta content="http://imgnews.naver.net/image/origin/018/2018/04/15/4079339.jpg" property="og:image"/>
    <meta content="[이데일리 김혜미 기자] 삼성전자(005930)의 전략 스마트폰 ‘갤럭시S9·S9+(플러스)’가 미국 대표 소비자 전문지 평가 1위를 기록했다. 13일(현지시간) 컨슈머 리포트는 갤럭

In [68]:
article = soup.find(id='articleBodyContents').get_text().strip()
# article = soup.find(class_='wiki-heading-content').get_text().strip()

In [69]:
# 필요한 텍스트만 파싱
article_parsed= []

r = re.compile('.+@.+[.]\w+') # email 거르기
for i in soup.find(id='articleBodyContents').children:
    try:
        i = i.strip() # 바로 자식노드의 텍스트만
        if len(i)>10 and not r.match(i): # 주석 거르기
            article_parsed.extend(i.split('. '))
    except:
        pass
    
print(len(article_parsed))
# [w for w in article_parsed if w is not '']
article_parsed

16


['[이데일리 김혜미 기자] 삼성전자(005930)의 전략 스마트폰 ‘갤럭시S9·S9+(플러스)’가 미국 대표 소비자 전문지 평가 1위를 기록했다.',
 '13일(현지시간) 컨슈머 리포트는 갤럭시S9의 내구성과 속도, 사운드를 장점으로 거론하며 올해 최고 스마트폰으로 평가했다',
 '앞서 컨슈머리포트는 지난 2016년과 2017년 갤럭시S7과 갤럭시S8을 각각 최고 스마트폰으로 평가한 바 있다.',
 '평가에서 갤럭시S9과 갤럭시S9+는 각각 81점을 기록했으며, 전작인 갤럭시S8이 80점으로 그 뒤를 이었다',
 '갤럭시S8 액티브, 갤럭시S8+는 모두 79점을 받아 1~5위를 모두 삼성전자 제품이 싹쓸이했다.',
 '6·7위는 각각 79점을 받은 애플 아이폰8+, 아이폰8이 차지했으며 아이폰X은 78점으로 8위를 나타냈다.',
 '컨슈머 리포트의 실험 결과 갤럭시S9 시리즈는 특히 내구성과 속도, 사운드가 전작보다 크게 개선됐다.',
 '컨슈머 리포트는 내구성을 테스트하기 위해 100번의 텀블러 낙하 실험을 실시했는데, 갤럭시S8과 갤럭시S8+는 50번 낙하 이후 작동하지 않았지만 갤럭시S9은 큰 이상이 없었던 것으로 나타났다',
 '다만 100번 떨어뜨린 이후에는 약간의 손상을 입는 데 그쳤다.',
 '또 갤럭시S9은 앱을 열거나 웹사이트를 로딩하는 속도가 이전보다 빨라진 것으로 평가됐다',
 '이는 이전보다 향상된 성능의 AP(애플리케이션 프로세서) 스냅드래곤845 영향인 것으로 보인다',
 '아울러 풀 사운드 재생이 가능한 스테레오 스피커 역시 높게 평가했다.',
 '컨슈머 리포트는 다만 갤럭시S9의 카메라로 촬영한 사진이 지나치게 뚜렷한 오버샤프닝(oversharpening) 현상이 나타나고, 이전보다 배터리 사용시간이 짧아진 것으로 나타나 갤럭시S9 시리즈의 전체 점수를 낮췄다고 설명했다',
 '오버샤프닝은 부자연스러운 후광과 과장된 대비효과로 오히려 사진의 품질을 떨어뜨린다는 것',
 '배터리는 일상적으로 하루 종일 사용할 수는 있지만, 전작보다

In [70]:
# 나무위키 파싱
article_parsed = [i.strip() for i in article.split('. ')]

In [361]:
# r = re.compile('.+@.+[.]\w+')
# res = r.match('honeybee@yna.co.kr')
# res.group()

---
## tokenizing, pos tagging

In [71]:
# pos tagging, tokenizing
from tqdm import tqdm_notebook
from konlpy.tag import Twitter
tagger = Twitter()

In [72]:
# *****pos tagging에 시간 좀 걸립니다******
def tokenize(doc):
    return ['/'.join(t) for t in tagger.pos(doc, norm=True, stem=True)]

sentences = []
for sentence in article_parsed:
    sentences.append(tokenize(sentence))

In [73]:
len(sentences)

13

In [74]:
# 'Noun','Verb','Adjective' 만 필터링
sentences2 = []
for sentence in sentences:
    sentences2.append([w for w in sentence 
                       if w.split('/')[1] in ['Noun','Verb','Adjective']])
print(sentences2)

[['오류/Noun', '우회/Noun', '하다/Verb', '위/Noun', '하다/Verb', '함수/Noun', '추가/Noun', '평점/Noun', '점/Noun', '위/Noun', '갤럭시/Noun', '시리즈/Noun', '위/Noun', '구성/Noun', '속도/Noun', '사운드/Noun', '전작/Noun', '대비/Noun', '크게/Noun', '개선/Noun', '돼다/Verb', '위/Noun', '애플/Noun', '제품/Noun', '아이폰/Noun', '아이폰/Noun', '이데일리/Noun', '김혜미/Noun', '기자/Noun', '삼성/Noun', '전자/Noun', '의/Noun', '전략/Noun', '스마트폰/Noun', '갤럭시/Noun', '플러스/Noun', '가다/Verb', '미국/Noun', '대표/Noun', '소비자/Noun', '전문/Noun', '평가/Noun', '위/Noun', '기록/Noun', '하다/Verb', '일/Noun', '현지/Noun', '시간/Noun', '컨슈머/Noun', '리포트/Noun', '갤럭시/Noun', '의/Noun', '구성/Noun', '속도/Noun', '사운드/Noun', '장점/Noun', '거론/Noun', '하다/Verb', '올해/Noun', '최고/Noun', '스마트폰/Noun', '평가/Noun', '하다/Verb'], ['앞서/Noun', '컨슈머/Noun', '리포트/Noun', '지난/Noun', '년/Noun', '년/Noun', '갤럭시/Noun', '과/Noun', '갤럭시/Noun', '각각/Noun', '최고/Noun', '스마트폰/Noun', '평가/Noun', '하다/Verb', '바/Noun', '있다/Adjective'], ['평가/Noun', '갤럭시/Noun', '과/Noun', '갤럭시/Noun', '늘다/Verb', '각각/Noun', '점/Noun', '기록/Noun', '하다/Verb', '전작/Noun'

In [75]:
from collections import Counter
bb=[]

for i in sentences2:
    bb.append(Counter(i))
    
bb

[Counter({'가다/Verb': 1,
          '개선/Noun': 1,
          '갤럭시/Noun': 3,
          '거론/Noun': 1,
          '구성/Noun': 2,
          '기록/Noun': 1,
          '기자/Noun': 1,
          '김혜미/Noun': 1,
          '대비/Noun': 1,
          '대표/Noun': 1,
          '돼다/Verb': 1,
          '리포트/Noun': 1,
          '미국/Noun': 1,
          '사운드/Noun': 2,
          '삼성/Noun': 1,
          '소비자/Noun': 1,
          '속도/Noun': 2,
          '스마트폰/Noun': 2,
          '시간/Noun': 1,
          '시리즈/Noun': 1,
          '아이폰/Noun': 2,
          '애플/Noun': 1,
          '오류/Noun': 1,
          '올해/Noun': 1,
          '우회/Noun': 1,
          '위/Noun': 5,
          '의/Noun': 2,
          '이데일리/Noun': 1,
          '일/Noun': 1,
          '장점/Noun': 1,
          '전략/Noun': 1,
          '전문/Noun': 1,
          '전자/Noun': 1,
          '전작/Noun': 1,
          '점/Noun': 1,
          '제품/Noun': 1,
          '최고/Noun': 1,
          '추가/Noun': 1,
          '컨슈머/Noun': 1,
          '크게/Noun': 1,
          '평가/Noun': 2,
        

In [82]:
a='apple is very good'
b='yes, i can do it'

aa= Counter(a)
bb= Counter(b)

In [84]:
print(aa)
print(bb)
print(aa & bb)

aaa={'a':1, 'b':2}
bbb={'a':2, 'c':3}
print(aaa&bbb)

Counter({' ': 3, 'p': 2, 'e': 2, 'o': 2, 'a': 1, 'l': 1, 'i': 1, 's': 1, 'v': 1, 'r': 1, 'y': 1, 'g': 1, 'd': 1})
Counter({' ': 4, 'i': 2, 'y': 1, 'e': 1, 's': 1, ',': 1, 'c': 1, 'a': 1, 'n': 1, 'd': 1, 'o': 1, 't': 1})
Counter({' ': 3, 'a': 1, 'e': 1, 'i': 1, 's': 1, 'y': 1, 'o': 1, 'd': 1})


TypeError: unsupported operand type(s) for &: 'dict' and 'dict'

In [28]:
q=1; qq=3
sum((bb[q] & bb[qq]).values()) / sum((bb[q] | bb[qq]).values())

0.09090909090909091

In [29]:
jj=[]
for i, el in enumerate(bb[:-1]):
    for i2 in range(i+1,len(bb)):
        sim = sum((bb[i] & bb[i2]).values()) / sum((bb[i] | bb[i2]).values())
        if sim>0: jj.append([i, sim, i2])
jj

[[0, 0.125, 1],
 [0, 0.025, 2],
 [0, 0.038461538461538464, 3],
 [0, 0.06, 4],
 [1, 0.05333333333333334, 2],
 [1, 0.09090909090909091, 3],
 [1, 0.08695652173913043, 4],
 [1, 0.08571428571428572, 5],
 [1, 0.025, 6],
 [1, 0.08333333333333333, 7],
 [1, 0.027777777777777776, 8],
 [1, 0.08695652173913043, 9],
 [1, 0.034482758620689655, 10],
 [1, 0.0625, 12],
 [1, 0.04477611940298507, 14],
 [2, 0.039473684210526314, 3],
 [2, 0.009615384615384616, 4],
 [2, 0.010526315789473684, 6],
 [2, 0.0125, 7],
 [2, 0.01098901098901099, 8],
 [2, 0.012658227848101266, 9],
 [2, 0.011904761904761904, 10],
 [2, 0.011363636363636364, 12],
 [2, 0.059322033898305086, 14],
 [3, 0.06382978723404255, 4],
 [3, 0.02702702702702703, 5],
 [3, 0.07894736842105263, 6],
 [3, 0.04, 7],
 [3, 0.041666666666666664, 9],
 [3, 0.0967741935483871, 12],
 [3, 0.029411764705882353, 14],
 [4, 0.12280701754385964, 5],
 [4, 0.08064516129032258, 6],
 [4, 0.0196078431372549, 7],
 [4, 0.016129032258064516, 8],
 [4, 0.02, 9],
 [4, 0.0181818

# gensim으로 모델링

In [145]:
from gensim import corpora

In [154]:
# https://radimrehurek.com/gensim/tut1.html
dictionary = corpora.Dictionary(sentences2)
print(dictionary)
pprint(dictionary.token2id)

Dictionary(231 unique tokens: ['것/Noun', '공동/Noun', '구축/Noun', '기대하다/Adjective', '내년/Noun']...)
{'가능해지다/Verb': 195,
 '가다/Verb': 164,
 '갖다/Verb': 165,
 '개발/Noun': 102,
 '개선/Noun': 33,
 '개정/Noun': 34,
 '개정안/Noun': 35,
 '거리/Noun': 55,
 '거치/Noun': 103,
 '거치다/Verb': 166,
 '건물/Noun': 56,
 '것/Noun': 0,
 '경매/Noun': 196,
 '계획/Noun': 197,
 '고/Noun': 198,
 '고시/Noun': 36,
 '공개/Noun': 199,
 '공동/Noun': 1,
 '공사/Noun': 104,
 '공청회/Noun': 200,
 '과기/Noun': 201,
 '과학기술/Noun': 37,
 '관련/Noun': 38,
 '관로/Noun': 57,
 '관리/Noun': 145,
 '광케이블/Noun': 58,
 '구리/Noun': 146,
 '구분/Noun': 167,
 '구역/Noun': 105,
 '구축/Noun': 2,
 '국사/Noun': 59,
 '굴착/Noun': 106,
 '금지/Noun': 226,
 '기계/Noun': 147,
 '기관/Noun': 148,
 '기대하다/Adjective': 3,
 '기존/Noun': 39,
 '기지국/Noun': 60,
 '길/Noun': 202,
 '내년/Noun': 4,
 '내용/Noun': 40,
 '넓다/Adjective': 61,
 '년/Noun': 5,
 '높다/Adjective': 62,
 '늘다/Verb': 63,
 '늘어나다/Verb': 149,
 '다른/Noun': 168,
 '달/Noun': 203,
 '대가/Noun': 169,
 '대도시/Noun': 204,
 '대비/Noun': 64,
 '대상/Noun': 107,
 '대역/Noun': 65,
 '대역폭/No

In [197]:
# document to bag of words
corpus = [dictionary.doc2bow(s) for s in sentences]
print(corpus[0])

bow = []
for c in corpus:
    a = [0]*(len(dictionary))
    for i in c:
        a[i[0]] = i[1]
    bow.append(a)
bow


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 3), (31, 1), (32, 1)]


[[1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  2,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  3,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [216]:
from sklearn.metrics import jaccard_similarity_score

jaccard_sim = []
for i, el in enumerate(bow[:-1]):
    for i2 in range(i+1,len(bow)):
        jaccard_sim.append([i, jaccard_similarity_score(bow[i], bow[i2]), i2])

In [217]:
jaccard_sim

[[0, 0.7835497835497836, 1],
 [0, 0.658008658008658, 2],
 [0, 0.6190476190476191, 3],
 [0, 0.7359307359307359, 4],
 [0, 0.6753246753246753, 5],
 [0, 0.7359307359307359, 6],
 [0, 0.8311688311688312, 7],
 [1, 0.6666666666666666, 2],
 [1, 0.6536796536796536, 3],
 [1, 0.7445887445887446, 4],
 [1, 0.6883116883116883, 5],
 [1, 0.6926406926406926, 6],
 [1, 0.8571428571428571, 7],
 [2, 0.5757575757575758, 3],
 [2, 0.6666666666666666, 4],
 [2, 0.5974025974025974, 5],
 [2, 0.5757575757575758, 6],
 [2, 0.7445887445887446, 7],
 [3, 0.6493506493506493, 4],
 [3, 0.5844155844155844, 5],
 [3, 0.5497835497835498, 6],
 [3, 0.696969696969697, 7],
 [4, 0.6796536796536796, 5],
 [4, 0.658008658008658, 6],
 [4, 0.8181818181818182, 7],
 [5, 0.5974025974025974, 6],
 [5, 0.7532467532467533, 7],
 [6, 0.7619047619047619, 7]]

In [188]:
# print(sentences2[0])
# print([dictionary[w] for w in dictionary.doc2idx(sentences2[0])])
# dictionary.doc2idx(sentences2[0])
dictionary[len(dictionary)-1]

'전재/Noun'

---
## compute rank

In [30]:
import math, random, csv, networkx as nx, operator
graph = nx.Graph()

# nodes = set([row[0] for row in jaccard_sim])
# edges = [(row[0], row[2]) for row in jaccard_sim]
nodes = set([row[0] for row in jj])
edges = [(row[0], row[2]) for row in jj]
num_nodes = len(nodes)
rank = 1/float(num_nodes)
graph.add_nodes_from(nodes, rank=rank)
graph.add_edges_from(edges)

In [31]:
V = float(len(graph))
s = 0.85
ranks = dict()
for key, node in graph.nodes(data=True):
    ranks[key] = node.get('rank')

In [32]:
for _ in range(10):
    for key, node in graph.nodes(data=True):
        rank_sum = 0.0
        curr_rank = node.get('rank')
        neighbors = graph[key]
        for n in neighbors: # for each neighbors, gather its pagerank
            if ranks[n] is not None:
                outlinks = len(graph.neighbors(n))
                rank_sum += (1 / float(outlinks)) * ranks[n]
        ranks[key] = ((1 - s) * (1/V)) + s*rank_sum
        
sorted_ranks = sorted(ranks.items(), key=operator.itemgetter(1), reverse=True)

for tuple in sorted_ranks: # print out the results
    print(tuple[0], tuple[1])    

4 0.09452429914971522
8 0.08181110392487556
14 0.08004916998791647
6 0.07997580996026846
1 0.0792218271986618
2 0.07351913042388611
12 0.07240861542437016
9 0.07238779660120837
7 0.07236565296116833
3 0.0676776218166642
5 0.06659128731831152
10 0.0665807847857918
0 0.03272878482958237
13 0.027198881705716126
11 0.02719337459795434


In [33]:
summary = []

# get lead sentence. first sentence
summary.append(article_parsed[0])

# print after sorting
for tuple in sorted(sorted_ranks[:2], key=lambda idx: idx[0]):
    summary.append(article_parsed[tuple[0]])
    
print(summary)

['대한민국 IT기업의 양대 축 중 하나대한민국에서 포털 사이트 점유율 22.4%(2위).[3] 점유율 출처', '많은 삽질에도 백색가전 덕택에 2위 자리를 지키는 LG전자처럼 카카오톡과 기존 다음 이용자들 덕택에 썩어도 준치인 기업2017년 5월 기준으로 재계서열 50위다.합병후인 이름 다음카카오에서 다음을 빼더니 계속 서비스 중 다음의 이름을 차차 지워가고 있다', '하지만 카카오뱅크도 초창기에는 잘 되다가 이후 3개월 정도가 지나서 갑자기 로그인 자체가 에러코드가 뜨면서 안 되는 등, 전혀 QA가 되지 않는 모습을 보였다']


---
# myTextRank.py


In [1]:
from myTextRank import *

In [2]:
p = textRank()

In [3]:
p.getNews(
    'http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0002814824'
)

['강원 양양군 현북면 잔교리에서 발생한 산불이 약 12시간 만인 20일 진화됐다.',
 '산림 당국은 20일 오전 4시 10분쯤 주불을 진화한 데 이어 오전 7시쯤 진화를 완료하고 뒷불감시 중이라고 밝혔다.',
 '산림 당국에 따르면 이 산불로 임야3㏊가 불에 탄 것으로 추정된다.',
 '양양 산불은 전날 오후 7시30분쯤 발생했다.',
 '불이 골짜리기를 타고 번진 탓에 오후 11시30분 쯤에는 인근 마을 주민 3명이 잔교리 마을회관에 대피하기도 했다.',
 '소방당국은 진화인력 358명과 소방펌프 등 장비 21대를 투입해 진화에 나섰지만, 해가 진 뒤 불이 나 헬기 투입이 불가능하고, 바람이 강하게 불어 진화 작업에 어려움을 겪었다.',
 '산림당국은 오전 5시 44분 쯤 일출과 동시에 진화헬기 9대, 군 4대, 소방 1대 등을 추가로 투입, 총 387명의 진화인력과 41대의 장비를 동원해 진화 작업에 나섰다.',
 '산림 당국 관계자는 "진화인력이 현장에 남아 잔불 정리를 철저히 할 것"이라며 "현재 건조한 날씨와 강한 바람이 지속하고 있는 상태로 산불위험이 큰 상황이어서 불씨 취급 등에 주의해 달라"고 당부했다.',
 '산림당국은 뒷불감시와 함께 정확한 피해면적과 화재원인을 조사할 방침이다.',
 '한편 18일 오후 4시쯤 동부전선 비무장지대 북측 구역에서 발생해 군사분계선(MDL)을 넘어 남하한 산불 진화작업도 재개됐다.',
 '일출과 동시에 산림청 헬기 4대가 투입돼 진화 중이다.',
 '군은 군사정전위원회의 승인을 받아 19일 오후 4시께부터 산림청 헬기를 투입, 진화작업을 벌여 80%를 진화한 상태다',
 '현재까지 군 피해 상황은 없는 것으로 알려졌다.',
 '이날 현재 강원 동해안과 북부·중부 산지에는 건조주의보가 발효 중이다.',
 '▶모바일에서 만나는 중앙일보',
 'and JTBC Content Hub Co., Ltd',
 '무단 전재 및 재배포 금지']

In [4]:
p.setGraph()

In [5]:
summ = p.getSummary()
print(summ)

['강원 양양군 현북면 잔교리에서 발생한 산불이 약 12시간 만인 20일 진화됐다.', '산림 당국은 20일 오전 4시 10분쯤 주불을 진화한 데 이어 오전 7시쯤 진화를 완료하고 뒷불감시 중이라고 밝혔다.', '산림 당국 관계자는 "진화인력이 현장에 남아 잔불 정리를 철저히 할 것"이라며 "현재 건조한 날씨와 강한 바람이 지속하고 있는 상태로 산불위험이 큰 상황이어서 불씨 취급 등에 주의해 달라"고 당부했다.']


In [6]:
p.getKeyword(5)

['진화', '시', '산불', '투입', '오후']

In [34]:
p.sorted_key_ranks
p.pos_tagged_noun
# print(p.word_count)


q = '오늘 어디에서 산불났어?'
# q = '오늘 산불 왜 났어?'
qq =['/'.join(t) for t in tagger.pos(q, norm=True, stem=True) 
     if t[1] in ['Noun','Verb','Adjective']]
cc = Counter(qq)
print(qq, '\n')


# jaccard sim 계산
ans = []
for idx, c in enumerate(p.word_count):
    sim = sum((c & cc).values()) / sum((c | cc).values())
    if sim>0: ans.append([idx, sim])

ans = sorted(ans, key=lambda i: i[1], reverse=True)

for i in ans:
    print(p.article_parsed[i[0]], i[1])

['오늘/Noun', '어디/Noun', '산불/Noun', '나다/Verb'] 

양양 산불은 전날 오후 7시30분쯤 발생했다. 0.09090909090909091
산림 당국에 따르면 이 산불로 임야3㏊가 불에 탄 것으로 추정된다. 0.06666666666666667
강원 양양군 현북면 잔교리에서 발생한 산불이 약 12시간 만인 20일 진화됐다. 0.058823529411764705
한편 18일 오후 4시쯤 동부전선 비무장지대 북측 구역에서 발생해 군사분계선(MDL)을 넘어 남하한 산불 진화작업도 재개됐다. 0.045454545454545456
산림 당국 관계자는 "진화인력이 현장에 남아 잔불 정리를 철저히 할 것"이라며 "현재 건조한 날씨와 강한 바람이 지속하고 있는 상태로 산불위험이 큰 상황이어서 불씨 취급 등에 주의해 달라"고 당부했다. 0.02702702702702703


In [7]:
# p.net_keyword
p.sorted_key_ranks

[('진화/Noun', 0.03799149447481222),
 ('시/Noun', 0.024914837336783065),
 ('산불/Noun', 0.024545768775321962),
 ('투입/Noun', 0.018072322449509924),
 ('오후/Noun', 0.01698581980678144),
 ('헬기/Noun', 0.01602412564757881),
 ('등/Noun', 0.01577176529518872),
 ('일/Noun', 0.015376860201876548),
 ('당국/Noun', 0.014795200945796498),
 ('분/Noun', 0.01439321351933999),
 ('발생/Noun', 0.014248291285896613),
 ('불/Noun', 0.013448885903009081),
 ('인력/Noun', 0.013163109493287957),
 ('현재/Noun', 0.012851929501396566),
 ('것/Noun', 0.01222080850491645),
 ('작업/Noun', 0.011946006655862986),
 ('군/Noun', 0.011806771613990397),
 ('바람/Noun', 0.01179983640915629),
 ('교리/Noun', 0.010848544228826381),
 ('잔/Noun', 0.010846980781872139),
 ('산림/Noun', 0.01066119343975823),
 ('쯤/Noun', 0.01034316468215761),
 ('장비/Noun', 0.009682078318297656),
 ('피해/Noun', 0.009495430250580257),
 ('상황/Noun', 0.009483666787789093),
 ('소방/Noun', 0.009406403854349769),
 ('중이/Noun', 0.009356448865391455),
 ('강원/Noun', 0.009132776718514478),
 ('오전/Noun

---

In [7]:
import nltk
from nltk.collocations import BigramCollocationFinder
# bigram_measures = nltk.collocations.BigramAssocMeasures()

In [20]:
a = '카카오는 인공지능(AI) 기술을 이용해 기사 선호 지역 등을 분석해 콜 성사 가능성이 큰 택시와 승객을 맞춰줄 수 있다는 입장이다.'
b = '카카오가 유료 호출 서비스인 ‘스마트호출’ 효과가 미미하자 목적지 미공개 방식을 철회했다'
f = BigramCollocationFinder.from_words(a.split(' '), window_size=5)
f2 = BigramCollocationFinder.from_words(b.split(' '), window_size=5)

# f.from_words(b.split(' '))

In [24]:
ff = f.ngram_fd; ff2 = f2.ngram_fd
print(len(ff), len(ff2))
len(ff+ff2)

66 34


100

In [26]:
'ff' in globals()

True

In [17]:
k= []
for i in f.ngram_fd:
    k.append([i[0], f.ngram_fd[i], i[1]])

In [6]:
p.getKeyword()

[('택시/Noun', 0.04106802339310693),
 ('목적지/Noun', 0.03221117821572837),
 ('수/Noun', 0.030501212464322396),
 ('호출/Noun', 0.028106065788589867),
 ('기사/Noun', 0.023083244454213593),
 ('사용자/Noun', 0.019986439034954288),
 ('카카오/Noun', 0.019684156051953793),
 ('원/Noun', 0.018008182370623186),
 ('승객/Noun', 0.017245460981993067),
 ('기능/Noun', 0.015871610035184492)]

In [24]:
from konlpy.tag import Kkma
from konlpy.corpus import kolaw
from konlpy.utils import pprint
from nltk import collocations


measures = collocations.BigramAssocMeasures()
doc = kolaw.open('constitution.txt').read()

In [28]:
# tagged_words = Kkma().pos(doc)
len(tagged_words)

10053

In [19]:
import asyncio
import requests

async def main(loop):
#     loop = asyncio.get_event_loop()
    future1 = loop.run_in_executor(None, requests.get, 'http://www.google.com')
    future2 = loop.run_in_executor(None, requests.get, 'http://www.naver.com')
    global response1
    response1 = await future1
    response2 = await future2
#     print(response1.text)
#     print(response2.text)

# loop = asyncio.get_event_loop()
r = loop.run_until_complete(main(loop))

In [50]:
# import sys
# from pprint import pprint
import re
import math, random, csv, networkx as nx, operator
from nltk.collocations import BigramCollocationFinder

# from gensim.summarization import summarize
# from gensim.summarization import keywords
# from gensim.summarization.textcleaner import split_sentences

import requests
from bs4 import BeautifulSoup

# pos tagging, tokenizing
from konlpy.tag import Twitter
tagger = Twitter()
from collections import Counter

import asyncio

"""
page rank class
"""

class textRank:
    async def getNews(self, url):
        self.url = url
        loop = asyncio.get_event_loop()  
        asyncio.set_event_loop(loop)
        s = requests.Session()
        task = loop.run_in_executor(None, s.get, self.url)
        text = await task
        soup = BeautifulSoup(text.text, 'html.parser')


        # save news title
        self.title = soup.find(id='articleTitle').get_text().strip()

        # 필요한 텍스트만 파싱
        self.article_parsed= []

        r = re.compile('.+@.+[.]\w+') # email 거르기
        for i in soup.find(id='articleBodyContents').children:
            try:
                i = i.strip() # 바로 자식노드의 텍스트만
                if len(i)>10 and not r.match(i): # 주석 거르기
                    self.article_parsed.extend(i.split('. '))
            except:
                pass
        
        return self.article_parsed


In [51]:
p = textRank()

In [52]:
url = "http://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0002814824"
# p.getNews(url)

In [53]:
loop = asyncio.get_event_loop()
print('1111111')
print(loop.run_until_complete(p.getNews(url)))
print('2222222')

1111111
['강원 양양군 현북면 잔교리에서 발생한 산불이 약 12시간 만인 20일 진화됐다.', '산림 당국은 20일 오전 4시 10분쯤 주불을 진화한 데 이어 오전 7시쯤 진화를 완료하고 뒷불감시 중이라고 밝혔다.', '산림 당국에 따르면 이 산불로 임야3㏊가 불에 탄 것으로 추정된다.', '양양 산불은 전날 오후 7시30분쯤 발생했다.', '불이 골짜리기를 타고 번진 탓에 오후 11시30분 쯤에는 인근 마을 주민 3명이 잔교리 마을회관에 대피하기도 했다.', '소방당국은 진화인력 358명과 소방펌프 등 장비 21대를 투입해 진화에 나섰지만, 해가 진 뒤 불이 나 헬기 투입이 불가능하고, 바람이 강하게 불어 진화 작업에 어려움을 겪었다.', '산림당국은 오전 5시 44분 쯤 일출과 동시에 진화헬기 9대, 군 4대, 소방 1대 등을 추가로 투입, 총 387명의 진화인력과 41대의 장비를 동원해 진화 작업에 나섰다.', '산림 당국 관계자는 "진화인력이 현장에 남아 잔불 정리를 철저히 할 것"이라며 "현재 건조한 날씨와 강한 바람이 지속하고 있는 상태로 산불위험이 큰 상황이어서 불씨 취급 등에 주의해 달라"고 당부했다.', '산림당국은 뒷불감시와 함께 정확한 피해면적과 화재원인을 조사할 방침이다.', '한편 18일 오후 4시쯤 동부전선 비무장지대 북측 구역에서 발생해 군사분계선(MDL)을 넘어 남하한 산불 진화작업도 재개됐다.', '일출과 동시에 산림청 헬기 4대가 투입돼 진화 중이다.', '군은 군사정전위원회의 승인을 받아 19일 오후 4시께부터 산림청 헬기를 투입, 진화작업을 벌여 80%를 진화한 상태다', '현재까지 군 피해 상황은 없는 것으로 알려졌다.', '이날 현재 강원 동해안과 북부·중부 산지에는 건조주의보가 발효 중이다.', '▶모바일에서 만나는 중앙일보', 'and JTBC Content Hub Co., Ltd', '무단 전재 및 재배포 금지']
2222222


In [37]:
# p.article_parsed
requests.get(url).text

'<!DOCTYPE HTML> \r\n<html lang="ko"> \r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="viewport" content="width=1106" />\r\n<title>양양 산불, 3㏊ 태우고 진화 완료…DMZ 북측 산불은 진화 중 : 네이버 뉴스</title>\r\n\n\n\t\n\t\n\t\t\n\t\n\n\n\t\n\t\n\t\n\t\t\n\t\t\n\t\t\n\t\n\n<meta property="me2:post_tag"\t\tcontent="중앙일보 : 네이버뉴스"/>\n<meta property="me2:category1"\t\tcontent="중앙일보"/>\n<meta property="me2:category2"\t\tcontent="사회"/>\n<meta property="me2:image"\t\t\tcontent="http://imgnews.naver.net/image/origin/025/2018/04/20/2814824.jpg"/>\n\n<meta property="og:title"\t\t\tcontent="양양 산불, 3㏊ 태우고 진화 완료…DMZ 북측 산불은 진화 중"/>\n<meta property="og:type"\t\t\tcontent="article"/>\n<meta property="og:url"\t\t\t\tcontent="http://news.naver.com/main/read.nhn?mode=LSD&mid=sec&oid=025&aid=0002814824&sid1=001"/>\n<meta property="og:image"\t\t\tcontent="http://imgnews.naver.net/image/origin/025/2018/04/20/2814824.jpg"/>\n<meta property="og:description"\t\tcontent="강원 양